In [1]:
from tqdm import tqdm
import os
import boto3
from dotenv import load_dotenv, find_dotenv
import threading
from queue import Queue
import pandas as pd
import json

In [2]:
# parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))

# print(parent_dir)
# # Encontra o arquivo .env no diretório pai
# dotenv_path = find_dotenv()

# Carrega as variáveis de ambiente do arquivo .env encontrado
load_dotenv()

True

In [4]:
with open('../data/people_and_pet.json', 'r') as f:
    dict_people_pet = json.load(f)

In [6]:
session = boto3.Session(
    aws_access_key_id       = os.getenv('aws_access_key_id'),  
    aws_secret_access_key   = os.getenv('aws_secret_access_key'),
    region_name             = os.getenv('AWS_DEFAULT_REGION'), 
    aws_session_token       = os.getenv('aws_session_token')
)

In [7]:
resource = session.resource("s3")

In [8]:
def directory_exists(bucket_name, directory_name):
    try:
        resource.head_object(Bucket=bucket_name, Key=directory_name)
        return True
    except:
        return False

In [9]:
import os

file_list = []
dir_atual = os.getcwd()
dir_destino = os.path.abspath(f'{os.pardir}/inteligencia_artificial/train/')

os.chdir(dir_destino)

arquivos_jpg = [f for f in os.listdir() if f.endswith('.jpg')]
fotos = set(arquivos_jpg)

In [10]:
len(fotos)

75501

In [11]:
bucket_name = 'pilha-nuvem-tcc-sptech-bucket'
num_threads = 20
q = Queue()
s3 = resource.Bucket(bucket_name)

def send_image_to_s3(md5_hash, folder, pet_name):
    image_with_extension = f"{md5_hash}.jpg"
    image_path = f"{dir_destino}/{image_with_extension}"
    with open(image_path, 'rb') as data:
        s3.put_object(Key = f"{folder}/{pet_name}_{image_with_extension}", Body=data)
    return True

In [13]:
for user_id, pets in tqdm(dict_people_pet.items()):
    user_folder = f"train/{user_id}"
    if not directory_exists(bucket_name = bucket_name, directory_name = user_folder):
        for pet in pets:
            if pet['image']:
                q.put((pet['image'], f"{user_folder}/{pet['id']}", pet['animal_name']))
                # send_image_to_s3(pet.image, f"{user_folder}/{pet.id}", pet.animal_name)

def send_images(queue: Queue, progress_bar: tqdm):
    while not queue.empty():
        pet_image, folder, pet_name = queue.get()
        try:
            send_image_to_s3(pet_image, folder, pet_name)
            progress_bar.update()
        except Exception as error:
            print(f"Erro ao enviar imagem {pet_image}: {error}")
        queue.task_done()

with tqdm(total=q.qsize()) as progress_bar:
    for i in range(num_threads):
        t = threading.Thread(target=send_images, args=(q, progress_bar))
        t.start()

    # esperar até que todas as fotos sejam enviadas
    q.join()

  0%|          | 0/134693 [00:00<?, ?it/s]